# Spoken Digit Challenge
This is the first challenge of our Speech and Machine Learning Workshop. Here we will use the [FSDD][] Free Spoken Digit Dataset to build different models and recognize the digits from speech.   

** Note: ** Make sure that your dataset is in the correct folder - if there´s something not working for you, feel free to ask.

* 1500 recordings in total (150 per digit)
* 8kHz sampling rate
* 3 speakers
* English 
* File format: {digit\_label}\_{speaker\_name}\_{index}.wav <br> (e.g. "4\_jackson\_16.wav")

[FSDD]: https://github.com/Jakobovski/free-spoken-digit-dataset  

In [ ]:
# Import the relevant modules to be used later
import glob
import os
import librosa, librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram

# Config matplotlib for inline plotting
%matplotlib inline